In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

import mlflow

In [ ]:
model_name = ""

In [3]:
def create_bert():
    pass

In [4]:
def create_custom_nn():
    pass

In [ ]:
bag_of_words_type = {"count-vectorizer": CountVectorizer, "tfidf": TfidfVectorizer}

def create_logreg_pipeline(bag_of_words: str):
    
    BoW = bag_of_words_type[bag_of_words]
    
    pass

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

In [ ]:
models = {
    "logistic-regression": create_logreg_pipeline,
    "bert": create_bert,
    "neural-network": create_custom_nn
}


model = models[model_name]

In [ ]:
with mlflow.start_run():
    mlflow.set_tags("model", model_name)
    mlflow.log_params()

    model.fit(X_train, y_train)
    
    
    y_pred = model.predict(X_test)
    
    score = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", score)